In [ ]:
import os
import numpy as np
from pydng.core import RPICAM2DNG
from time import time
import uuid
from io import BytesIO
from skimage.feature import blob_log
from matplotlib import pyplot as plt
from matplotlib.patches import Circle, Polygon

%matplotlib inline
figsize = (20,20)

def _plot(img, vmax=0):
    fig = plt.figure(figsize=figsize)
    ax = fig.add_subplot(111)
    if vmax > 0:
        ax.imshow(img, cmap='gray', interpolation='none', vmax=vmax)
    else:
        ax.imshow(img, cmap='gray', interpolation='none')
    return ax

def _circle(ax, pt, rad=6,c=(1,0,0)):
    circ = Circle(pt,rad,facecolor=c)
    ax.add_patch(circ)
    
def _triangle(ax, tri, c=(1,0,0)):
#     print("got triangle", tri, "color",c)
      poly = Polygon(tri, True, color=c, fill=False)
      ax.add_patch(poly)
    
    
def openRaw(image_file):
    with open(image_file, 'rb') as fh:
        buf = BytesIO(fh.read())
    rpicam = RPICAM2DNG()
    raw = rpicam.__extractRAW__(buf)
    raw = np.array(raw, dtype=np.float)
    #0.2125 R + 0.7154 G + 0.0721 B

    carray = 0.2125*raw[::2,::2] + 0.7154 * (raw[1::2,::2] + raw[::2,1::2]) / 0.5 + 0.0721 * raw[1::2,1::2]
    carray = carray[200:-200,400:-500]
    carray /= np.max(carray)
    return carray    

def getConstellation(pts):
    '''
    
    '''
    # first get all possible triangles
    print("Number stars", len(pts), pts[0])
    N = len(pts)
    triangles = []
    tpoints = []
    
    for tt in range(N):
        for ttt in range(tt+1,N):
            for tttt in range(ttt+1,N):
                # the triangle now is pt tt,ttt,tttt
                pt1 = pts[tt][1::-1]
                pt2 = pts[ttt][1::-1]
                pt3 = pts[tttt][1::-1]
                l1 = np.sqrt(np.sum((pt1-pt2)**2))
                l2 = np.sqrt(np.sum((pt2-pt3)**2))
                l3 = np.sqrt(np.sum((pt3-pt1)**2))
                tri = [l1,l2,l3]
                tri.sort()
                triangles.append(np.array(tri))                
                tpoints.append(np.array((pt1,pt2,pt3)))
#     print("len",len(triangles),triangles[0])
    return triangles, tpoints

def compareConstelations(tri1,tri2):
    ptlist = []
    for cnt,triangle in enumerate(tri1):
        small = 100000000
        idx = None
        for cc,triangle2 in enumerate(tri2):
            distance = np.sqrt(np.sum((triangle-triangle2)**2))
            if distance < small:
                idx = cc
                small = distance
        ptlist.append((small,cnt,idx))
    ptlist.sort()
    return ptlist[:100]

class match(object):
    def __init__(self, blob):
        self.x = blob[0]
        self.y = blob[1]
        self.matchlist = [(self.x,self.y)]
        self.key = uuid.uuid1()
            
    def isMatch(self, blob):
        xdst = blob[0]
        ydst = blob[1]
        distance = np.sqrt((self.x - xdst)**2 + (self.y - ydst)**2)
        if distance < 5:
            self.x = xdst
            self.y = ydst
            self.matchlist.append((self.x,self.y))
            return True
        return False

def findMatching(blobs, blob_dst):
    retdict = {}
    # long brute force, need a better way
    for bl2 in blob_dst:
        found = False
        for blkey in blobs:
            bl1 = blobs[blkey]
            check = bl1.isMatch(bl2)
            if check:
                found = True
                break
        if not found:
            nmatch = match(bl2)
            retdict[nmatch.key] = nmatch
    return {**blobs, **retdict}

def stackImagesRaw(file_list):
    stacked_image = None
    first_blobs = {}
    nM = np.zeros((3,3))
    for count,file in enumerate(file_list):
#         print(file)
        
        imageF = openRaw(file)
        blobs_log = blob_log(imageF, min_sigma=2, max_sigma=10, num_sigma=5, threshold=.02)

        if stacked_image is None:
            # Save keypoints for first image
            stacked_image = imageF
            first_blobs = findMatching(first_blobs,blobs_log)
            ax = _plot(imageF)
        else:
            first_blobs = findMatching(first_blobs,blobs_log)
            if count % 30 == 0:
                print("Found matches", len(first_blobs), "count", count)
#             src_pts = []
#             dst_pts = []
#             for match in matches:
#                 y1,x1,y2,x2 = match
                
#                 src_pts.append((x1,y1))
#                 dst_pts.append((x2,y2))

#             src = np.array(src_pts, dtype=np.float32).reshape(-1,1,2)
#             dst = np.array(dst_pts, dtype=np.float32).reshape(-1,1,2)
            
#             if len(src) < 10:
#                 continue

#             Estimate perspective transformation
#             M, mask = cv2.findHomography(src, dst, cv2.RANSAC, 5.0)
#             nM += M
#             print("Diff", M - nM/count)
#             w, h = imageF.shape
#             print("M",M)
#             stacked_image = cv2.warpPerspective(stacked_image, M, (h, w))
#             first_blobs = blobs_log
#             if count == len(file_list) - 1:
#                 ax = _plot(imageF)
#             for match in matches:
#                 y1,x1,y2,x2 = match
#                 _circle(ax,(x1,y1),c='red',rad=3)
#                 _circle(ax,(x2,y2),c='yellow',rad=1)
#             stacked_image += imageF
    print("Total matches", len(first_blobs))
    valid = 0
    good = len(file_list) / 3
    for matchkey in first_blobs:
        
        if len(first_blobs[matchkey].matchlist) > good:
            valid += 1
            for match in first_blobs[matchkey].matchlist:
                y1,x1 = match
                _circle(ax,(x1,y1),c='red',rad=1)
    print("Total valid", valid)
    
    return stacked_image / count

In [ ]:
image_folder = "."
file_list = os.listdir(image_folder)
file_list = [os.path.join(image_folder, x) for x in file_list if x.startswith('testa_')]
file_list.sort()
# print(file_list)
stacked_image = stackImagesRaw(file_list[:])
# _plot(stacked_image)